In [2]:
import os
directory = r"C:\Users\ATsareva\Desktop\LARP"
os.chdir(directory)
from vk_tools import *

In [3]:
import pandas as pd
import time
import sqlite3

In [4]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///ki.db')

In [5]:
def get_group_id(vk_club):
    if vk_club.find('club') != -1:
        group_id = vk_club[4:]
    elif vk_club.find('event') != -1:
        group_id = vk_club[5:]
    elif vk_club.find('public') != -1:
        group_id = vk_club[6:]
    else:
        group_id = vk_club
    try:
        int(group_id)
    except: 
        group_id = vk_club
    return group_id

In [16]:
df = pd.read_csv('ki_games.csv')

#only larps from SPb&LO
spb_only = (df.sub_region_id==1)|(df.sub_region_id==2)
#delete konvents, dances, competitions, festivals
larps_only = (df.type!=5)&(df.type!=6)&(df.type!=10)&(df.type!=13)

spb_larps = df[(larps_only)&(spb_only)].copy()
spb_larps.loc['begin'] = pd.to_datetime(spb_larps.begin)

In [17]:
groups = spb_larps.vk_club.dropna().values

In [18]:
group_ids = []
for vk_club in groups:
    group_id = get_group_id(vk_club)
    if group_id not in group_ids: group_ids.append(group_id)

groupinfo = get_groupinfo(group_ids)

In [20]:
gid_list = []
for group in groupinfo:
    if group['id'] not in gid_list: gid_list.append(group['id'])

In [38]:
def get_members_df(members):
    list_of_members = []
    for member in members:
        list_of_members.append(member['id'])
    members_df = pd.DataFrame(columns = ['users', group_id])
    members_df.users = list_of_members
    members_df[group_id] = 1
    
    return members_df

In [81]:
subscriptions = pd.DataFrame(columns = ['users'])
n = 1
for group_id in gid_list:
    members = get_members_df(get_all_members(group_id))
    subscriptions = pd.merge(subscriptions, members, how='outer',on='users')
    print('Обработано %s из %s' % (n, len(gid_list)))
    n+=1

Обработано 1 из 93
Обработано 2 из 93
Обработано 3 из 93
Обработано 4 из 93
Обработано 5 из 93
Обработано 6 из 93
Обработано 7 из 93
Обработано 8 из 93
Обработано 9 из 93
Обработано 10 из 93
Обработано 11 из 93
Обработано 12 из 93
Обработано 13 из 93
Обработано 14 из 93
Обработано 15 из 93
Обработано 16 из 93
Обработано 17 из 93
Обработано 18 из 93
Обработано 19 из 93
Обработано 20 из 93
Обработано 21 из 93
Обработано 22 из 93
Обработано 23 из 93
Обработано 24 из 93
Обработано 25 из 93
Обработано 26 из 93
Обработано 27 из 93
Обработано 28 из 93
Обработано 29 из 93
Обработано 30 из 93
Обработано 31 из 93
Обработано 32 из 93
Обработано 33 из 93
Обработано 34 из 93
Обработано 35 из 93
Обработано 36 из 93
Обработано 37 из 93
Обработано 38 из 93
Обработано 39 из 93
Обработано 40 из 93
Обработано 41 из 93
Обработано 42 из 93
Обработано 43 из 93
Обработано 44 из 93
Обработано 45 из 93
Обработано 46 из 93
Обработано 47 из 93
Обработано 48 из 93
Обработано 49 из 93
Обработано 50 из 93
Обработан

In [82]:
profiles = pd.DataFrame()
s = 0
e = 1000
while s < len(subscriptions)+1:
    add_profiles = get_profiles(subscriptions.users.values[s:e])
    profiles = profiles.append(add_profiles)
    s+=1000
    e+=1000

In [83]:
profiles = get_age_from_profile(profiles)

In [84]:
profiles.rename(columns={'id': 'users'}, inplace=True)
subscriptions = pd.merge(subscriptions, profiles, how='outer', on='users')

In [120]:
barrayar = subscriptions[['users', 'city', 'sex', 'age', 121128769]].copy()
barrayar[121128769] = barrayar[121128769].fillna(0)
barrayar = barrayar.dropna()

In [100]:
from sklearn.tree import DecisionTreeClassifier

In [121]:
barrayar['city_code'] = pd.factorize(barrayar['city'])[0]

In [122]:
barrayar.sex = barrayar.sex.astype(int)
barrayar.age = barrayar.sex.astype(int)
barrayar[121128769] = barrayar[121128769].astype(int)


In [125]:
y = barrayar[121128769]

In [127]:
cities = barrayar.city

In [129]:
barrayar.drop(['city'], axis=1, inplace=True)

In [130]:
X_train, X_holdout, y_train, y_holdout = train_test_split(barrayar.values, y, test_size=0.3,
random_state=17)

In [131]:
tree = DecisionTreeClassifier(max_depth=5, random_state=17)

In [135]:
from sklearn.metrics import accuracy_score

tree_pred = tree.predict(X_holdout)
accuracy_score(y_holdout, tree_pred)

1.0

In [139]:
knn = KNeighborsClassifier(n_neighbors=10)

In [141]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [142]:
knn_pred = knn.predict(X_holdout)
accuracy_score(y_holdout, knn_pred)

0.98863636363636365

0    1056
dtype: int64